dùng scipy.signal tạo hàm truyền động cơ dc
\begin{equation}
    G(s) = \frac{K}{\tau s + 1}
\end{equation}

In [ ]:
import numpy as np 
import matplotib.pyplot as plt
import scipy.signal as signal

#----------------------------------
# Parameters
#----------------------------------

k = 1.0
tau = 0.5

num = [k]
den = [tau, 1]
plant = signal.TransferFunction(num, den)
